# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments  \
0  Jun 16 2023 10:07AM  264883        21   9578437   
1  Jun 16 2023 10:00AM  264880        15  30017264   
2  Jun 16 2023 10:00AM  264880        15  30017265   
3  Jun 16 2023 10:00AM  264880        15  30017266   
4  Jun 16 2023 10:00AM  264880        15  30017267   
5  Jun 16 2023  9:59AM  264881        15   9578412   
6  Jun 16 2023  9:59AM  264881        15   9578413   
7  Jun 16 2023  9:59AM  264881        15   9578414   
8  Jun 16 2023  9:59AM  264881        15   9578415   
9  Jun 16 2023  9:59AM  264881        15   9578416   

                          Customer ShipmentStatus  
0                NBTY Global, Inc.       Released  
1            Alliance Pharma, Inc.       Released  
2            Alliance Pharma, Inc.       Released  
3            Alliance Pharma, Inc.       Released  
4            Alliance Pharma, Inc.       Released  
5  Brookfield Pharmaceuticals, LLC       Released  
6  Brookfield Pharmaceuticals, LLC       Released  
7  Brookfield Pharmaceuticals, LLC       Released  
8  Brookfield Pharmaceuticals, LLC       Released  
9  Brookfield Pharmaceuticals, LLC       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
16  264878       Released          1
17  264879       Released          1
18  264880       Released          4
19  264881       Released         23
20  264883       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
264878                NaN        NaN       1.0
264879                NaN        NaN       1.0
264880                NaN        NaN       4.0
264881                NaN        NaN      23.0
264883                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264825                0.0        0.0       1.0
264828               11.0        1.0       0.0
264835                0.0        1.0       0.0
264844                0.0        0.0       1.0
264845                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
264825                  0          0         1
264828                 11          1         0
264835                  0          1         0
264844                  0          0         1
264845                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               264825          0          0         1
1               264828         11          1         0
2               264835          0          1         0
3               264844          0          0         1
4               264845          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               264825                            1
1               264828        11         1         
2               264835                   1         
3               264844                            1
4               264845                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.
1   Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.
5   Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC
28  Jun 16 2023  9:49AM  264879        15                            Lonza
29  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.
30  Jun 16 2023  9:35AM  264876        10                 Eywa Pharma Inc.
31  Jun 16 2023  9:28AM  264877        10                   Citieffe, Inc.
32  Jun 16 2023  9:01AM  264875        10                        Hush Hush
34  Jun 16 2023  8:52AM  264873        10                      Emerginnova
36  Jun 16 2023  8:51AM  264872        10              Digital Brands, LLC

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.   
1  Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.   
2  Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC   
3  Jun 16 2023  9:49AM  264879        15                            Lonza   
4  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.   
5  Jun 16 2023  9:35AM  264876        10                 Eywa Pharma Inc.   
6  Jun 16 2023  9:28AM  264877        10                   Citieffe, Inc.   
7  Jun 16 2023  9:01AM  264875        10                        Hush Hush   
8  Jun 16 2023  8:52AM  264873        10                      Emerginnova   
9  Jun 16 2023  8:51AM  264872        10              Digital Brands, LLC   

  Completed Executing Released  
0                            1  
1                            4  
2                           23  
3                            1  
4                            1  
5                            1  
6                            1  
7                   2           
8                            2  
9                            2

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.   
1  Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.   
2  Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC   
3  Jun 16 2023  9:49AM  264879        15                            Lonza   
4  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.   
5  Jun 16 2023  9:35AM  264876        10                 Eywa Pharma Inc.   
6  Jun 16 2023  9:28AM  264877        10                   Citieffe, Inc.   
7  Jun 16 2023  9:01AM  264875        10                        Hush Hush   
8  Jun 16 2023  8:52AM  264873        10                      Emerginnova   
9  Jun 16 2023  8:51AM  264872        10              Digital Brands, LLC   

  Released Executing Completed  
0        1                      
1        4                      
2       23                      
3        1                      
4        1                      
5        1                      
6        1                      
7                  2            
8        2                      
9        2

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.   
1  Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.   
2  Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC   
3  Jun 16 2023  9:49AM  264879        15                            Lonza   
4  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.   

  Released Executing Completed  
0        1                      
1        4                      
2       23                      
3        1                      
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.   
1  Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.   
2  Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC   
3  Jun 16 2023  9:49AM  264879        15                            Lonza   
4  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.   

   Released  Executing  Completed  
0       1.0        NaN        NaN  
1       4.0        NaN        NaN  
2      23.0        NaN        NaN  
3       1.0        NaN        NaN  
4       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                         Customer  \
0  Jun 16 2023 10:07AM  264883        21                NBTY Global, Inc.   
1  Jun 16 2023 10:00AM  264880        15            Alliance Pharma, Inc.   
2  Jun 16 2023  9:59AM  264881        15  Brookfield Pharmaceuticals, LLC   
3  Jun 16 2023  9:49AM  264879        15                            Lonza   
4  Jun 16 2023  9:45AM  264878        15            Alliance Pharma, Inc.   

   Released  Executing  Completed  
0       1.0        0.0        0.0  
1       4.0        0.0        0.0  
2      23.0        0.0        0.0  
3       1.0        0.0        0.0  
4       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         2913452      26.0        5.0       11.0
12          264866       0.0        1.0        0.0
15         1324371      29.0        1.0        0.0
19          264825       1.0        0.0        0.0
20          264850       1.0        0.0        0.0
21          264883       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  2913452      26.0        5.0       11.0
1        12   264866       0.0        1.0        0.0
2        15  1324371      29.0        1.0        0.0
3        19   264825       1.0        0.0        0.0
4        20   264850       1.0        0.0        0.0
5        21   264883       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      26.0        5.0       11.0
1        12       0.0        1.0        0.0
2        15      29.0        1.0        0.0
3        19       1.0        0.0        0.0
4        20       1.0        0.0        0.0
5        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   26.0
1        12  Released    0.0
2        15  Released   29.0
3        19  Released    1.0
4        20  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   19   20   21
Status                                   
Completed  11.0  0.0   0.0  0.0  0.0  0.0
Executing   5.0  1.0   1.0  0.0  0.0  0.0
Released   26.0  0.0  29.0  1.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   19   20   21
0          Completed  11.0  0.0   0.0  0.0  0.0  0.0
1          Executing   5.0  1.0   1.0  0.0  0.0  0.0
2           Released  26.0  0.0  29.0  1.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   19   20   21
0  Completed  11.0  0.0   0.0  0.0  0.0  0.0
1  Executing   5.0  1.0   1.0  0.0  0.0  0.0
2   Released  26.0  0.0  29.0  1.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()